In [1]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_from_disk
from collections import Counter
from itertools import compress

dataset_path = "/workspaces/Implementation-of-Neural-Machine-Translation-of-Rare-Words-with-Subword-Units-on-Azerbaijani/data_source"
ds = load_from_disk(dataset_path=dataset_path)

/home/codespace/.local/share/virtualenvs/Implementation-of-Neural-Machine-Translati-2BUKKnR4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.model_selection import train_test_split

raw_data = ds["train"].to_pandas()
X = raw_data["partial_text"].map(lambda text: text.lower())
y = raw_data["gold_ending"].map(lambda text: text.lower())

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1/0.9, random_state=42)


X_train_val.reset_index(inplace=True, drop=True)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
X_val.reset_index(inplace=True, drop=True)

y_train_val.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)
y_val.reset_index(inplace=True, drop=True)

In [3]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors

# Initialize a tokenizer
tokenizer = Tokenizer(models.BPE())

# Customize pre-tokenization and decoding
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)

# And then train
initial_aze_alphabet = list(set(' '.join(X_train_val)))


trainer = trainers.BpeTrainer(
    vocab_size=20000, min_frequency=2,
    initial_alphabet=initial_aze_alphabet,
    special_tokens=["<bos>", "<eos>", "<pad>", "<mask>"])

tokenizer.train_from_iterator(X_train_val.map(lambda sentence: "<bos> " + sentence + " <eos>"), trainer=trainer)
tokenizer.save(path=os.path.join(os.path.join(os.getcwd(), "tokenizer_files/aze_nsp.json")))

In [4]:
sample_sentence = "<bos> Bürc Xəlifə hal-hazırda 111 dünyada ən hündür göydələndir <eos>"
tokenizer.decode(tokenizer.encode(sample_sentence.lower()).ids)

' bürc xəlifə hal-hazırda 111 dünyada ən hündür göydələndir '

In [5]:
from huffman_tree import build_huffman_tree_from_tokenizer
save_dir = os.path.join(os.getcwd(), "huffman_save/")
build_huffman_tree_from_tokenizer(tokenizer, X_train_val, save_dir=save_dir)